In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder
import pandas as pd
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [2]:
train = train.drop(['월','요일','강설량(mm)','적설량(cm)','풍향','짙은안개','번개','진눈깨비','서리','연기/연무','눈날림'],axis=1)
test = test.drop(['월','요일','강설량(mm)','적설량(cm)','풍향','짙은안개','번개','진눈깨비','서리','연기/연무','눈날림'],axis=1)

In [3]:
print(sorted(train['범죄발생지'].unique()))
print(sorted(test['범죄발생지'].unique()))

['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']
['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [5]:
x_train = train.drop(['ID'], axis = 1)
# y_train = train['TARGET']

x_test = test.drop('ID', axis = 1)

In [6]:
ordinal_features = ['범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(x_train[feature])
    x_train[feature] = le.transform(x_train[feature])

    # x_train데이터에서 존재하지 않았던 값이 x_test 데이터에 존재할 수도 있습니다.
    # 따라서 x_test 데이터를 바로 변형시키지 않고 고윳값을 확인후 x_test 데이터를 변환합니다.
    for label in np.unique(x_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_test[feature] = le.transform(x_test[feature])

In [11]:
from pycaret import classification
classification.setup(data=x_train,target='TARGET')

ModuleNotFoundError: No module named 'pycaret'

In [12]:
best = classification.compare_models(n_select=5,fold=10)

NameError: name 'classification' is not defined

In [11]:
probas = []
for model in best:
    probas.append(model.predict_proba(x_test))

probas = np.array(probas)
pred = [np.argmax(i) for i in 0.4*probas[0]+0.3*probas[1]+0.3*probas[2]] # np.array(probas).mean(axis=0)

In [12]:
sub = pd.read_csv('./sample_submission.csv')
sub.set_index('ID',inplace=True)
sub['TARGET'] = pred
sub.to_csv('./answer.csv')